### Churn @ Robinhood
#### Load Libraries and Data

In [1]:
from numba import cuda

try:
    print("Available GPUs:", cuda.gpus)
except cuda.CudaSupportError as e:
    print("CUDA Error:", e)

Available GPUs: <Managed Device 0>


In [2]:
import cudf as cf
equity_df_raw = cf.read_csv('./data/equity_value_data.csv')
features_df_raw = cf.read_csv('./data/features_data.csv')

equity_df = equity_df_raw.copy()
features_df = features_df_raw.copy()

In [3]:
equity_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1119158 entries, 0 to 1119157
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype
---  ------        --------------    -----
 0   timestamp     1119158 non-null  object
 1   close_equity  1119158 non-null  float64
 2   user_id       1119158 non-null  object
dtypes: float64(1), object(2)
memory usage: 72.6+ MB


In [4]:
features_df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 5584 entries, 0 to 5583
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   risk_tolerance                5584 non-null   object
 1   investment_experience         5584 non-null   object
 2   liquidity_needs               5584 non-null   object
 3   platform                      5584 non-null   object
 4   time_spent                    5584 non-null   float64
 5   instrument_type_first_traded  5584 non-null   object
 6   first_deposit_amount          5584 non-null   float64
 7   time_horizon                  5584 non-null   object
 8   user_id                       5584 non-null   object
dtypes: float64(2), object(7)
memory usage: 896.8+ KB


#### a). What percentage of users have churned in the data?
A user is *churned* when their equity falls below 10 usd for 28 consecutive calendar days or longer having perviously been at least 10 usd

In [5]:
# to ensure timestamp is datetime
equity_df['timestamp'] = cf.to_datetime(equity_df['timestamp']).copy() 

# Sort Values by user_id and timestamp
equity_df = equity_df.sort_values(['user_id', 'timestamp']).copy()

# Flag close_equity below 10 dollars
equity_df['below_10'] = (equity_df['close_equity'] < 10).astype(int).copy()

# Compute rolling 28-day windows for each user
equity_df['below_10_28d'] = equity_df.groupby('user_id')['below_10'].rolling(window=28, min_periods=28).sum().reset_index(0,drop=True).copy()

# Identify churn accounts
equity_df['churn'] = (equity_df['below_10_28d'] == 28).astype(int).copy()

# Ensure customer previously had >= 10 dollars
# Group by 'user_id' and find the max close_equity for each user
user_max_equity = equity_df.groupby('user_id')['close_equity'].max().reset_index()
user_max_equity.rename(columns={"close_equity": "max_equity"}, inplace=True)

print("user_max_equity:")
print(user_max_equity.head())

# Merge back into the original DataFrame
equity_df['user_id'] = equity_df['user_id'].astype(str)
user_max_equity['user_id'] = user_max_equity['user_id'].astype(str)

equity_df = equity_df.merge(user_max_equity, on="user_id", how="left")

print("equity_df after merge:")
print(equity_df.head())
print(equity_df.columns)

# Add a flag for users who had close_equity >= 10 at some point
equity_df['above_10_before'] = (equity_df['max_equity'] >= 10.0).astype(int)

# Filter churned customers
churned_users = equity_df.loc[(equity_df['churn'] == 1) & (equity_df['above_10_before'] == 1), 'user_id'].unique().copy()

# Calculate churn percentage
total_users = equity_df['user_id'].nunique()
churned_percentage = (len(churned_users) / total_users) * 100

print(f'Percent churned: {churned_percentage:.2f}%')

user_max_equity:
                            user_id  max_equity
0  5fd4a01d324f8d20eaa2c43809e9f948    748.3000
1  d10f9f7ac787d96a3ae3880a95e90468   7222.8105
2  c06642abdd9bc364efc158190d537f6f   9995.0000
3  88905c55ea6570090f5bf731ea6e2f68   1856.3828
4  fb35f2d4f7bd6c9e4085856fe14e8d89   1001.1900
equity_df after merge:
   timestamp  close_equity                           user_id  below_10  \
0 2017-07-11       72.5348  00440034cc4152bfb01b30f5c381c4e3         0   
1 2017-07-12       72.2840  00440034cc4152bfb01b30f5c381c4e3         0   
2 2017-07-13       74.3668  00440034cc4152bfb01b30f5c381c4e3         0   
3 2017-07-14       72.4600  00440034cc4152bfb01b30f5c381c4e3         0   
4 2017-07-17       71.9420  00440034cc4152bfb01b30f5c381c4e3         0   

   below_10_28d  churn  max_equity  
0             0      0        86.8  
1             0      0        86.8  
2             0      0        86.8  
3             0      0        86.8  
4             0      0        86.8  
Index